# 第5回 その1: 主成分分析
第2回のレポート課題で使用した，5教科テストデータに対して主成分分析を行ってみましょう。


## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/05"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/05/
# 現在のフォルダの中身を表示
%ls

`gokyouka.csv`というデータが表示されていることを確認してください。

## ステップ1: データの読み込み
まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

`gokyouka.csv` を読み込みます。  
このデータは第2回のレポート課題で使用した，5教科のテストの点数を示したデータ(report02_input.csv)です。

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('gokyouka.csv', encoding='SHIFT-JIS')
# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

# numpy用データ(ndarray型) に変換する。
X = csv_data.to_numpy()

# データのサンプル数と次元数を得る。
(num_samples, num_dimensions) = np.shape(X)
print('Nunber of samples: ' + str(num_samples))
print('Number of dimensions: ' + str(num_dimensions))

データが `X` に格納されました。
`Number of samples` はデータ内のサンプル数，つまり生徒数です。  
`Number of dimensions` は1サンプルの次元数，つまり教科数です。  
つまりこのデータは5次元ベクトルが100サンプル格納されたデータということになります。  
この5次元ベクトルデータを主成分分析により2次元に圧縮し，プロットすることを試みます。

## ステップ2: データの標準化  
(今回のデータは単位が揃っていますが)一般的なデータでは，各次元の値の単位（スケール）が揃っていないことが多いです（例えば身長(cm)と体重(kg)のデータ）。  
そこでスケールを合わせるための前処理として，各次元の平均と分散で正規化を行います。  

In [ ]:
# 次元毎の平均と分散
mu = np.mean(X, axis=0)
std = np.std(X, axis=0)

# 標準化
X_norm = (X - mu) / std

## ステップ3: 分散共分散行列の計算  
標準化したデータの分散共分散行列を計算します。  
分散共分散行列は主成分分析の計算過程で必要な他，  
次元毎のばらつきや，次元間の相関関係を知ることにも使用できます。  

分散共分散行列の計算には numpy の <font color="Red"> **`cov`**</font> 関数を使用します。  
ただし，`cov`関数に入力する前に，データを転置し，[次元数, サンプル数]にしておく必要があります。


In [ ]:
# 分散共分散行列の計算
# X.T とすることで転置してから cov に入力する点に注意
cov = np.cov(X_norm.T, bias=True)
print(cov)

分散共分散の対角成分は，次元毎の分散値，つまり教科ごとの点数の分散です。  
非対角成分は，次元間，つまり２教科間の相関係数を表します。  
例えば 1行2列目の0.85243365は，1次元目と2次元目，つまり国語と英語の相関係数を表します。  
3行4列目の0.88124546 は，3次元目と4次元目，つまり数学と物理の相関係数を表します。  
分散共分散行列は対称行列（cov = cov.T）です。  
（国語と英語の相関 = 英語と国語の相関）

## ステップ4: 分散共分散行列の固有値分解
主成分分析を実行するためには，分散共分散行列を固有値分解し，固有値の大きい順に固有ベクトルを並べ替えます。  
固有値と固有ベクトルの計算には，numpy の <font color="Red"> **`linalg.eig`**</font> 関数を使用します。  
また，並べ替えには numpy の <font color="Red"> **`argsort`**</font> 関数を使用します。  
ただし argsort は小さい順に並び替えるため，argsortを使って小さい順に並べた結果に`[::-1]`とつけることで，さらに逆順（=大きい順）に並び替えています。

In [ ]:
# 固有値 eig_val と 固有ベクトル eig_vec を計算
eig_val, eig_vec =np.linalg.eig(cov)
# データによっては固有値分解結果が複素数になる場合があるため，実部のみ取得
eig_val = np.real(eig_val)
eig_vec = np.real(eig_vec)

# 固有値を大きい順に並び替える
order = np.argsort(eig_val)[::-1]
eig_val = eig_val[order]
eig_vec = eig_vec[:,order]
print('Eigen values:')
print(eig_val)
print('Eigen vectors:')
print(eig_vec)

固有ベクトル（Eigen vectors）は，要素数5の列ベクトルが横に5個並んでいます。  
つまり，固有値3.26に対応する固有ベクトルは[-0.36, -0.36, -0.50, -0.50, -0.49]です。

## ステップ4: 主成分分析による次元圧縮
第一主成分および第二主成分の固有ベクトルを使って，データを二次元に圧縮します。

In [ ]:
X_pca = np.dot(X_norm, eig_vec[:,:2])

# データのサンプル数と次元数を得る。
(num_samples_pca, num_dimensions_pca) = np.shape(X_pca)
print('Nunber of samples: ' + str(num_samples_pca))
print('Number of dimensions: ' + str(num_dimensions_pca))

主成分分析により，データが5次元から2次元に圧縮されました。  

次に，横軸を第一主成分，縦軸を第二主成分として，2次元に圧縮したデータをプロットします。

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(X_pca[:,0], X_pca[:,1], color='b')
plt.xlabel('1st principal component')
plt.ylabel('2nd principal component')
plt.show()

## ステップ5: 主成分の考察  
ステップ4で，5次元のデータを2次元平面上にプロットすることはできました。  
このとき，横軸と縦軸がどういう意味を持っているのかについて考察しましょう。  

まず，第一主成分と第二主成分の固有ベクトル，つまり圧縮時に用いる各次元の係数を見てみましょう。

In [ ]:
print('Coefficients for the 1st principal component')
print(eig_vec[:,0])

print('Coefficients for the 2nd principal component')
print(eig_vec[:,1])

第一主成分は，  
-0.36\*国語 - 0.36\*英語 - 0.50\*数学 - 0.50\*物理 -0.49\*化学  
として計算されています。  
全ての教科に負の係数がかけられているため，テストの点が良いほど，第一主成分は負の方向に大きくなります。  
また，文系科目よりも理系科目の方がやや比重が大きく計算されていることが分かります。  

第二主成分は，
-0.61\*国語 - 0.60\*英語 + 0.26\*数学 + 0.29\*物理 + 0.33\*化学  
として計算されています。  
文系科目には負の係数，理系科目には正の係数がかけられているため，テストの点が文系科目に偏るほど第二主成分は負の方向へ，逆に理系科目に偏るほど第二主成分は正の方向へ大きくなることにな。

テストの総合点が高い程赤く，低い程青くなるように散布図の色を変えてプロットしてみましょう。

In [ ]:
# 各サンプルのテストの総合点を計算
color = np.sum(X,axis=1)

plt.figure(figsize=(11,5))
plt.scatter(X_pca[:,0], X_pca[:,1], c=color, cmap='jet')
plt.xlabel('1st principal component')
plt.ylabel('2nd principal component')
plt.colorbar(label='Total score of the 5 subjects')
plt.show()

上の図によると，合計点が低い（色が青い）ほど右に，高い（色が赤い）ほど左に分布していることが分かります。  
このことから，**第一主成分（横軸）はテストの総合的な出来と関係性が強い軸が作られている**ことが分かります。

次に，文系科目の平均点と理系科目の平均点の割合を計算し，理系科目の割合が高い程赤く，文系科目の割合が高いほど青くなるように散布図の色を変えてプロットしてみます。

In [ ]:
# 理系科目の平均点 / 文系科目の平均点
color = np.mean(X[:,2:],axis=1) / np.mean(X[:,:2], axis=1)

plt.figure(figsize=(11,5))
plt.scatter(X_pca[:,0], X_pca[:,1], c=color, cmap='jet')
plt.xlabel('1st principal component')
plt.ylabel('2nd principal component')
plt.colorbar(label='bunkei / rikei')
plt.show()

上の図によると，文系科目の点数の割合が高い（色が青い）ほど下に，理系科目の点数の割合が高い（色が赤い）ほど上に分布していることが分かります。  
このことから，**第二主成分（縦軸）は文系-理系の得意科目傾向と関係が強い軸が作られている**ことが分かります。  

横軸「総合的なテストの出来」と縦軸「文系-理系の得意科目傾向」の二軸で今回のデータを俯瞰すると，データの性質が色々と見えてきます。例えば  
* 総合的なテストの出来は，「良い（左側）」，「普通（中央）」，「悪い（右側）」の３グループに分かれる。  
* 「普通」の集団は，文系-理系の偏りが広いのに対して，「良い」および「悪い」の集団では，あまり文系-理系の偏りが広くない。  

と言ったことがわかります。  
このことから，今回のテストは出来が「良い」「普通」「悪い」グループにはっきりと分かれており，「良い」集団はどの教科も点数は高く，「悪い」集団はどの教科も点数が低い，ということが分かります。

### 補足
第一主成分は総合点が高い程，負の方向に大きくなっているため，直感と異なるかもしれません。  
主成分分析側からすると，入力されたデータの意味，つまり各データがテストの点で，大きいほど価値が高いというような情報は一切知りません。  
あくまでデータの相対的な散らばり具合を保存するように分析をしているだけですので，必ずしも正負が直感に沿った関係になるとは限りません。また，データの相対的な位置関係に意味があるのであって，横軸の-4～4，縦軸の-4～2といった値自体には特に意味はありません。  

主成分分析は単にデータの散らばり具合を保存する軸を抽出しているだけであり，軸の意味は解析者が考える必要があります。
今回の例では，第一主成分(横軸)にテストの総合点，第二主成分(縦軸)に理系/文系傾向といった，直感的に意味が分かりやすい軸が得られましたが，必ずしも理解しやすい軸が常に得られる訳では無いという点には注意してください。  
（事実，今回の例でも第三主成分以降はよく分からない軸になっています。）  